In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test_df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
sample_submission.head()

#Adjust
---
  + if Image -> score > Metadata -> score:
    1. 이미지 분석을 이용하여 바로 ***Image -> Pawpularity*** 모델 구축
    2. Metadata와 Pawpularity score를 이용하여 ***Metadata -> Pawpularity*** 모델 구축
    3. 이미지 분석을 이용하여 ***Image -> Metadata*** 모델 구축
    4. 이미지로 예측한 Pawpularity score에 + α(Metadata -> score 일정 비율 반영)

  + elif Image -> score > Metadata -> score:
    1. 이미지 분석을 이용하여 바로 ***Image -> Pawpularity*** 모델 구축
    2. Metadata와 Pawpularity score를 이용하여 ***Metadata -> Pawpularity*** 모델 구축
    3. 이미지 분석을 이용하여 ***Image -> Metadata*** 모델 구축
    4. Metadata로 예측한 Pawpularity socre에 + α(***Image -> score*** 일정 비율 반영)

  + else:
    1. 이미지와 Meata를 CNN 등 방법을 이용하여 ***Image -> Metadata*** 모델 구축
    2. Metadata와 Pawpularity score를 이용하여 ***Metadata -> Pawpularity*** 모델 구축
    3. 주어진 이미지를 이용하여 ***Image -> Metadata***(생성)
    4. 생성된 Metadata를 이용하여 ***Metadata -> Pawpularity*** 예측

# Metadata -> score
sklearn을 이용하여 model을 불러온다. 
-> 직접 model을 제작해서 하는 방향으로 해보지

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
train, test = train_test_split(train_df, test_size=0.3, random_state=777)

train_x = train[train.columns[2:13]]
train_y = train[train.columns[13]] # Pawpularity

test_x = test[test.columns[2:13]]
test_y = test[test.columns[13]]
 
real_test_x = test_df[test_df.columns[2:13]]

In [ ]:
train_x.info()

In [ ]:
train_y

In [ ]:
model = LogisticRegression(solver='liblinear')
model.fit(train_x, train_y) 
predict = model.predict(test_x)

print('The accuracy of the Logistic Regression is', metrics.accuracy_score(predict, test_y))

In [ ]:
model = LogisticRegression(solver='liblinear')
model.fit(train_x, train_y)
predict = model.predict(real_test_x)

sample_submission['Id'] = test_df[test_df.columns[0]]
sample_submission['Pawpularity'] = predict
sample_submission.to_csv('submission.csv',index=False)

In [ ]:
sample_submission.head()